In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import torch.nn.functional as F

In [3]:
df_away=pd.read_csv('/content/drive/MyDrive/data/Train_Data/train_away_team_statistics_df.csv').drop(['LEAGUE', 'TEAM_NAME'], axis=1)
df_home=pd.read_csv('/content/drive/MyDrive/data/Train_Data/train_home_team_statistics_df.csv').drop(['LEAGUE', 'TEAM_NAME'], axis=1)
df_home.columns = [('HOME_' +str(col) ) if col != 'ID' else 'ID' for col in df_home.columns ]
df_joined=df_away.join(df_home.set_index('ID'), on='ID')
df_joined=df_joined.fillna(0.0)
df_final=df_joined.sort_values(by=['ID']).drop(['ID'], axis=1)
df_final.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/Train_Data/train_away_team_statistics_df.csv'

In [261]:
df_result=pd.read_csv('/content/drive/MyDrive/data/Train_Data/Y_train.csv')

df_result_final=df_result.drop(['ID'], axis=1)
df_result.head()

,ID,HOME_WINS,DRAW,AWAY_WINS
0,0,0,0,1
1,1,0,1,0
2,2,0,0,1
3,3,1,0,0
4,4,0,1,0


In [262]:
X_train, X_test, y_train, y_test = train_test_split(df_final, df_result_final, test_size=0.1, random_state=42)

In [263]:
X_train

,TEAM_SHOTS_TOTAL_season_sum,TEAM_SHOTS_INSIDEBOX_season_sum,TEAM_SHOTS_OFF_TARGET_season_sum,TEAM_SHOTS_ON_TARGET_season_sum,TEAM_SHOTS_OUTSIDEBOX_season_sum,TEAM_PASSES_season_sum,TEAM_SUCCESSFUL_PASSES_season_sum,TEAM_SAVES_season_sum,TEAM_CORNERS_season_sum,TEAM_FOULS_season_sum,...,HOME_TEAM_YELLOWCARDS_5_last_match_std,HOME_TEAM_REDCARDS_5_last_match_std,HOME_TEAM_OFFSIDES_5_last_match_std,HOME_TEAM_ATTACKS_5_last_match_std,HOME_TEAM_PENALTIES_5_last_match_std,HOME_TEAM_SUBSTITUTIONS_5_last_match_std,HOME_TEAM_BALL_SAFE_5_last_match_std,HOME_TEAM_DANGEROUS_ATTACKS_5_last_match_std,HOME_TEAM_INJURIES_5_last_match_std,HOME_TEAM_GOALS_5_last_match_std
73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,2.0,...,4.0,0.0,2.0,0.0,0.0,0.0,4.0,5.0,5.0,0.0
8890,4.0,4.0,6.0,2.0,4.0,4.0,4.0,4.0,3.0,3.0,...,1.0,0.0,0.0,2.0,5.0,4.0,3.0,3.0,8.0,5.0
2009,7.0,7.0,6.0,6.0,5.0,7.0,7.0,5.0,4.0,3.0,...,1.0,0.0,4.0,2.0,0.0,6.0,0.0,0.0,0.0,4.0
7983,5.0,3.0,5.0,5.0,8.0,5.0,5.0,10.0,3.0,8.0,...,0.0,0.0,0.0,1.0,0.0,6.0,2.0,1.0,10.0,2.0
1795,6.0,9.0,9.0,4.0,3.0,3.0,3.0,8.0,3.0,10.0,...,4.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,5.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,6.0,6.0,5.0,6.0,7.0,7.0,7.0,4.0,4.0,4.0,...,5.0,0.0,5.0,2.0,0.0,4.0,0.0,4.0,8.0,10.0
5191,8.0,9.0,5.0,8.0,1.0,4.0,4.0,4.0,5.0,1.0,...,3.0,5.0,0.0,10.0,0.0,9.0,5.0,2.0,2.0,0.0
5390,5.0,6.0,7.0,4.0,3.0,5.0,5.0,3.0,4.0,0.0,...,2.0,0.0,0.0,7.0,5.0,4.0,7.0,6.0,3.0,4.0
860,1.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,...,6.0,0.0,0.0,9.0,0.0,10.0,0.0,3.0,0.0,10.0


In [264]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [265]:
device

device(type='cuda')

In [266]:
from torch.utils.data import Dataset, DataLoader
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(np.array(X_train)).to(torch.float32)
        self.y=torch.from_numpy(np.array(y_train)).to(torch.long)
        self.len=self.x.shape[0]
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [274]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(280, 4),
            nn.Sigmoid(),
            nn.Linear(4, 2),
            nn.Sigmoid(),
            nn.Linear(2, 3),
            nn.Softmax(),

        )

    def forward(self, x):
        return self.layers(x)

In [275]:
data_set=Data()
model = MLP()
model=model.to(device)

In [276]:
model.layers[0].weight.dtype

torch.float32

In [277]:
criterion=nn.CrossEntropyLoss().to(device)

In [278]:
trainloader=DataLoader(dataset=data_set,batch_size=10)


In [279]:
dataloader_iterator = iter(trainloader)
x, y = next(dataloader_iterator)

In [280]:
model(x.to(device))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([[0.2955, 0.3034, 0.4011],
        [0.2929, 0.3033, 0.4038],
        [0.2982, 0.3038, 0.3981],
        [0.2983, 0.3039, 0.3978],
        [0.2989, 0.3040, 0.3971],
        [0.2926, 0.3033, 0.4041],
        [0.3049, 0.3046, 0.3905],
        [0.2940, 0.3031, 0.4029],
        [0.2944, 0.3035, 0.4021],
        [0.2963, 0.3035, 0.4002]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [281]:
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [290]:
n_epochs=20
loss_list=[]

#n_epochs
for epoch in range(n_epochs):
    print(f'epoch {epoch}')
    for x, y in trainloader:


        #clear gradient
        optimizer.zero_grad()
        #make a prediction
        z=model(x.to(device))
        loss=criterion(z,y.to(torch.float32).to(device))
        # calculate gradients of parameters
        loss.backward()
        # update parameters
        optimizer.step()

        loss_list.append(loss.data)
    print(f'loss is {loss_list[-1]}')


epoch 0


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss is 1.0136668682098389
epoch 1
loss is 1.011407732963562
epoch 2
loss is 1.0092699527740479
epoch 3
loss is 1.0073151588439941
epoch 4
loss is 1.005576491355896
epoch 5
loss is 1.003978967666626
epoch 6
loss is 1.0024592876434326
epoch 7
loss is 1.001007080078125
epoch 8
loss is 0.9996092319488525
epoch 9
loss is 0.9982534646987915
epoch 10
loss is 0.99693763256073
epoch 11
loss is 0.9956653118133545
epoch 12
loss is 0.9944460391998291
epoch 13
loss is 0.9932878017425537
epoch 14
loss is 0.9921678304672241
epoch 15
loss is 0.9910244345664978
epoch 16
loss is 0.9898014068603516
epoch 17
loss is 0.9884900450706482
epoch 18
loss is 0.9871343374252319
epoch 19
loss is 0.9858014583587646


In [283]:
loss_list[-1]

tensor(1.0160, device='cuda:0')

In [291]:
test_data = torch.from_numpy(np.array(X_test)).to(torch.float32).to(device)
test_targets = torch.from_numpy(np.array(y_test)).to(torch.long).to(device)

In [292]:
model.eval()

MLP(
  (layers): Sequential(
    (0): Linear(in_features=280, out_features=4, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=4, out_features=2, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=2, out_features=3, bias=True)
    (5): Softmax(dim=None)
  )
)

In [293]:
with torch.no_grad():
    predicted_labels = model(test_data)




In [294]:
predicted_labels

tensor([[0.9335, 0.0322, 0.0343],
        [0.7425, 0.0816, 0.1759],
        [0.1815, 0.1148, 0.7037],
        ...,
        [0.5619, 0.1087, 0.3294],
        [0.2012, 0.1167, 0.6821],
        [0.9082, 0.0405, 0.0513]], device='cuda:0')

In [316]:
torch.sum(torch.argmax(predicted_labels, 1)==2)

tensor(383, device='cuda:0')

In [295]:
torch.sum(torch.argmax(predicted_labels, 1)==torch.argmax(test_targets, 1))/len(predicted_labels)

tensor(0.5150, device='cuda:0')

In [296]:
mse = mean_squared_error(test_targets.cpu().numpy(), predicted_labels.cpu().numpy())
r2 = r2_score(test_targets.cpu().numpy(), predicted_labels.cpu().numpy())
print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 0.23733721072278227
R2 Score: -0.1072353545325575


In [299]:
df_away_test=pd.read_csv('/content/drive/MyDrive/data/Test_Data/test_away_team_statistics_df.csv')
df_home_test=pd.read_csv('/content/drive/MyDrive/data/Test_Data/test_home_team_statistics_df.csv')
df_home_test.columns = [('HOME_' +str(col) ) if col != 'ID' else 'ID' for col in df_home_test.columns ]
df_joined_test=df_away_test.join(df_home_test.set_index('ID'), on='ID')
df_joined_test=df_joined_test.fillna(0.0)
df_final_test=df_joined_test.sort_values(by=['ID']).drop(['ID'], axis=1)

In [347]:
df_joined_test

,ID,TEAM_SHOTS_TOTAL_season_sum,TEAM_SHOTS_INSIDEBOX_season_sum,TEAM_SHOTS_OFF_TARGET_season_sum,TEAM_SHOTS_ON_TARGET_season_sum,TEAM_SHOTS_OUTSIDEBOX_season_sum,TEAM_PASSES_season_sum,TEAM_SUCCESSFUL_PASSES_season_sum,TEAM_SAVES_season_sum,TEAM_CORNERS_season_sum,...,HOME_TEAM_YELLOWCARDS_5_last_match_std,HOME_TEAM_REDCARDS_5_last_match_std,HOME_TEAM_OFFSIDES_5_last_match_std,HOME_TEAM_ATTACKS_5_last_match_std,HOME_TEAM_PENALTIES_5_last_match_std,HOME_TEAM_SUBSTITUTIONS_5_last_match_std,HOME_TEAM_BALL_SAFE_5_last_match_std,HOME_TEAM_DANGEROUS_ATTACKS_5_last_match_std,HOME_TEAM_INJURIES_5_last_match_std,HOME_TEAM_GOALS_5_last_match_std
0,12303,6.0,6.0,5.0,7.0,7.0,4.0,3.0,6.0,3.0,...,2.0,9.0,0.0,10.0,0.0,4.0,4.0,6.0,2.0,3.0
1,12304,4.0,4.0,4.0,4.0,3.0,8.0,8.0,4.0,7.0,...,9.0,10.0,3.0,1.0,8.0,3.0,8.0,1.0,0.0,2.0
2,12305,1.0,1.0,2.0,2.0,2.0,6.0,5.0,4.0,2.0,...,6.0,8.0,2.0,9.0,0.0,6.0,0.0,9.0,0.0,0.0
3,12306,6.0,9.0,6.0,9.0,4.0,4.0,4.0,1.0,10.0,...,4.0,5.0,2.0,3.0,8.0,0.0,2.0,2.0,2.0,3.0
4,12307,5.0,5.0,2.0,4.0,4.0,1.0,2.0,3.0,8.0,...,5.0,0.0,0.0,7.0,5.0,0.0,1.0,6.0,7.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25363,37666,8.0,8.0,6.0,10.0,7.0,8.0,8.0,10.0,3.0,...,2.0,6.0,2.0,1.0,5.0,6.0,6.0,5.0,2.0,0.0
25364,37667,7.0,4.0,4.0,9.0,8.0,7.0,6.0,3.0,9.0,...,3.0,0.0,2.0,8.0,0.0,0.0,4.0,8.0,10.0,10.0
25365,37668,3.0,4.0,2.0,3.0,1.0,1.0,1.0,3.0,4.0,...,7.0,0.0,8.0,4.0,0.0,7.0,0.0,2.0,0.0,3.0
25366,37669,5.0,5.0,5.0,3.0,4.0,3.0,3.0,3.0,5.0,...,5.0,0.0,4.0,7.0,8.0,5.0,6.0,4.0,5.0,4.0


In [301]:
df_test_torch=torch.from_numpy(np.array(df_final_test)).to(torch.float32)

In [303]:
test_result=model(df_test_torch.to(device))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [306]:
test_result[:10]

tensor([[0.6888, 0.0921, 0.2190],
        [0.1609, 0.1119, 0.7272],
        [0.9300, 0.0334, 0.0366],
        [0.2619, 0.1215, 0.6166],
        [0.1631, 0.1123, 0.7246],
        [0.9307, 0.0332, 0.0361],
        [0.3362, 0.1231, 0.5407],
        [0.2155, 0.1181, 0.6664],
        [0.5830, 0.1057, 0.3113],
        [0.5217, 0.1138, 0.3645]], device='cuda:0', grad_fn=<SliceBackward0>)

In [318]:
argmax=torch.argmax(test_result, 1)

In [326]:
result = F.one_hot(argmax, num_classes=3)

In [341]:
result_pd = pd.DataFrame(result.cpu().numpy())


In [343]:
result_pd.rename(columns={'0': 'HOME_WINS', '1': 'DRAW', '2': 'AWAY_WINS'})
result_pd.columns = ['HOME_WINS' ,'DRAW','AWAY_WINS']


In [345]:
result_pd.index=df_final_test.index

In [348]:
test_prediction=pd.concat([df_joined_test['ID'],result_pd], axis=1)


In [349]:
test_prediction

,ID,HOME_WINS,DRAW,AWAY_WINS
0,12303,1,0,0
1,12304,0,0,1
2,12305,1,0,0
3,12306,0,0,1
4,12307,0,0,1
...,...,...,...,...
25363,37666,1,0,0
25364,37667,0,0,1
25365,37668,1,0,0
25366,37669,1,0,0


In [351]:
test_prediction.to_csv('./mlp_benchmark_submission.csv', index=False)
